# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.14 which is incompatible.


The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726671e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.41999999061226845


### Hyperparameter set of the agent

In [6]:
# Necessary imports
import random

from agent import Agent

The following class will randomly sample a set of the hyperparameters of our agent. This randomly sampled set will be used to evaluate the agent for a small number of iterations. The score for each training will be saved and finally we will use the hyperparameters with highes score for our final training.   

In [7]:
class SampleHyperparameter(object):
    '''Returned randomly sampled hyperparameters.
        Return Type: Dictionary
    '''
    def __init__(self):
        self.TAU = [1e-3, 1e-5, 1e-1]     # for soft update of target parameters
        self.LR_ACTOR = [1e-4, 1e-2, 1e-3]              # learning rate of the actor 
        self.LR_CRITIC = [1e-4, 1e-2, 1e-3]             # learning rate of the critic
        self.WEIGHT_DECAY = [1e-1, 1e-2, 1e-3]             # L2 weight decay
        self.ACTOR_FC1_UNITS = [1024, 512, 256]        # Number of units for the layer 1 in the actor model
        self.ACTOR_FC2_UNITS = [512, 256, 128]        # Number of units for the layer 2 in the actor model
        self.CRITIC_FC1_UNITS = [1024, 512, 256]      # Number of units for the layer 1 in the critic model
        self.CRITIC_FC2_UNITS = [512, 256, 128]       # Number of units for the layer 2 in the critic model
        self.BN_MODE = [0, 1, 2]                  # Use Batch Norm. - 0=disabled, 1=BN before Activation, 2=BN after Activation (3, 4 are alt. versions of 1, 2)
        self.ADD_OU_NOISE = [True, False]          # Add Ornstein-Uhlenbeck noise
        
        self.return_list = {}

    def get_random_sample(self):
        self.return_list['tau'] = random.choice(self.TAU)
        self.return_list['lr_actor'] = random.choice(self.LR_ACTOR)
        self.return_list['lr_critic'] = random.choice(self.LR_CRITIC)
        self.return_list['wd'] = random.choice(self.WEIGHT_DECAY)
        architecture_index = random.choice([0,1,2])
        self.return_list['actor_fc1'] = self.ACTOR_FC1_UNITS[architecture_index]
        self.return_list['actor_fc2'] = self.ACTOR_FC2_UNITS[architecture_index]
        self.return_list['critic_fc1'] = self.CRITIC_FC1_UNITS[architecture_index]
        self.return_list['critic_fc2'] = self.CRITIC_FC2_UNITS[architecture_index]
        self.return_list['bn_mode'] = random.choice(self.BN_MODE)
        self.return_list['ou_noise'] = random.choice(self.ADD_OU_NOISE)
        
        return self.return_list
        

In [8]:
#test hyperparameter random sampling
hyperparameter_obj = SampleHyperparameter()
for _ in range(5):
    print(hyperparameter_obj.get_random_sample())

{'tau': 0.001, 'lr_actor': 0.001, 'lr_critic': 0.0001, 'wd': 0.1, 'actor_fc1': 512, 'actor_fc2': 256, 'critic_fc1': 512, 'critic_fc2': 256, 'bn_mode': 2, 'ou_noise': True}
{'tau': 0.1, 'lr_actor': 0.0001, 'lr_critic': 0.0001, 'wd': 0.1, 'actor_fc1': 512, 'actor_fc2': 256, 'critic_fc1': 512, 'critic_fc2': 256, 'bn_mode': 1, 'ou_noise': False}
{'tau': 0.1, 'lr_actor': 0.01, 'lr_critic': 0.01, 'wd': 0.1, 'actor_fc1': 256, 'actor_fc2': 128, 'critic_fc1': 256, 'critic_fc2': 128, 'bn_mode': 0, 'ou_noise': True}
{'tau': 1e-05, 'lr_actor': 0.0001, 'lr_critic': 0.001, 'wd': 0.1, 'actor_fc1': 256, 'actor_fc2': 128, 'critic_fc1': 256, 'critic_fc2': 128, 'bn_mode': 0, 'ou_noise': False}
{'tau': 0.1, 'lr_actor': 0.0001, 'lr_critic': 0.01, 'wd': 0.001, 'actor_fc1': 512, 'actor_fc2': 256, 'critic_fc1': 512, 'critic_fc2': 256, 'bn_mode': 2, 'ou_noise': False}


In [12]:
#Fixed hyperparameters
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 128        # minibatch size
GAMMA = 0.99            # discount factor
MU = 0.                 # Ornstein-Uhlenbeck noise parameter
THETA = 0.15            # Ornstein-Uhlenbeck noise parameter
SIGMA = 0.2             # Ornstein-Uhlenbeck noise parameter

#Default settings
TAU = 1e-3              # for soft update of target parameters
LR_ACTOR = 1e-4         # learning rate of the actor 
LR_CRITIC = 1e-3        # learning rate of the critic
WEIGHT_DECAY = 0        # L2 weight decay
ACTOR_FC1_UNITS = 400   # Number of units for the layer 1 in the actor model
ACTOR_FC2_UNITS = 300   # Number of units for the layer 2 in the actor model
CRITIC_FCS1_UNITS = 400 # Number of units for the layer 1 in the critic model
CRITIC_FC2_UNITS = 300  # Number of units for the layer 2 in the critic model
BN_MODE = 0             # Use Batch Norm. - 0=disabled, 1=BN before Activation, 2=BN after Activation (3, 4 are alt. versions of 1, 2)
ADD_OU_NOISE = True     # Add Ornstein-Uhlenbeck noise

### DDPG training method

In [13]:
def ddpg(n_episodes=100, max_t=1000,
         state_size=state_size, action_size=action_size, random_seed=1234, 
         actor_fc1_units=ACTOR_FC1_UNITS, actor_fc2_units=ACTOR_FC2_UNITS,
         critic_fcs1_units=CRITIC_FCS1_UNITS, critic_fc2_units=CRITIC_FC2_UNITS,
         buffer_size=BUFFER_SIZE, batch_size=BATCH_SIZE, bn_mode=BN_MODE,
         gamma=GAMMA, tau=TAU, lr_actor=LR_ACTOR, lr_critic=LR_CRITIC, weight_decay=WEIGHT_DECAY,
         add_ounoise=ADD_OU_NOISE, mu=MU, theta=THETA, sigma=SIGMA):  
    
    # Instantiate the Agent
    agent = Agent(state_size=state_size,action_size=action_size, random_seed=random_seed,
                  actor_fc1_units=actor_fc1_units, actor_fc2_units=actor_fc2_units,
                  critic_fcs1_units=critic_fcs1_units, critic_fc2_units=critic_fc2_units,
                  buffer_size=buffer_size, batch_size=batch_size, bn_mode=bn_mode,
                  gamma=gamma, tau=tau,
                  lr_actor=lr_actor, lr_critic=lr_critic, weight_decay=weight_decay,
                  add_ounoise=add_ounoise, mu=mu, theta=theta, sigma=sigma)
                  
    scores_deque = deque(maxlen=100)
    scores = []

    print("\nStart training:")
    for i_episode in range(1, n_episodes+1):
        
        # Reset the env and get the state (Single Agent)
        env_info = env.reset(train_mode=True)[brain_name]     
        state = env_info.vector_observations[0]
        
        # Reset the DDPG Agent (Reset the internal state (= noise) to mean mu)
        agent.reset()
        
        # Reset the score 
        score = 0
        
        for t in range(max_t):
            action = agent.act(state)                   # select an action 

            env_info=env.step(action)[brain_name]        # send action to the environment
            next_state = env_info.vector_observations[0] # get next state (Single Agent)
            reward = env_info.rewards[0]                 # get reward (Single Agent)
            done = env_info.local_done[0]                # see if episode finished (Single Agent)
            
            # Save experience in replay memory, and use random sample from buffer to learn
            agent.step(state, action, reward, next_state, done)
            
            state = next_state
            score += reward
            if done:
                #print("Episode {} has terminated at step {}".format(i_episode, t))
                break 
        
        # Save scores and compute average score over last 100 episodes
        scores_deque.append(score)
        scores.append(score)
        avg_score = np.mean(scores_deque)
        
        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, avg_score, score), end="")
        if i_episode % 100 == 0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            # Early stop
            if avg_score > 30:
                print('\rEnvironment solved in {} episodes with an Average Score of {:.2f}'.format(i_episode, avg_score))
                return scores
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, avg_score))  
    return scores

### Method for visualization

In [14]:
def plot_training(scores):
    plt.plot(np.arange(1, len(scores)+1), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

### Searching for hyperparameters

When finished, you can close the environment.

In [ ]:
env.close()